In [1]:
from __future__ import print_function

import sys
import os
root = os.path.abspath('/home/mbeliaev/home/code/robust-l0/')
sys.path.append(root)

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import numpy as np
from tqdm import trange
import foolbox


import torchvision
import torchvision.transforms as transforms


from randomizedAblation import foolbox_utils
from randomizedAblation.prog_bar import progress_bar
import randomizedAblation.utils_color as utils_cifar
import randomizedAblation.utils as utils
import randomizedAblation.pytorch_cifar.models.resnet as resnet

# torch.cuda.empty_cache()

stty: 'standard input': Inappropriate ioctl for device


In [2]:
in_b = np.array([0,1])
beta = 100

out = in_b/(1/beta)
out += ((1-beta)/2)

print(out)

[-49.5  50.5]


In [3]:
def convert_state_dict(torch_path):
    # original saved file with DataParallel
    state_dict = torch.load(torch_path)
    # create new OrderedDict that does not contain `module.`
    from collections import OrderedDict
    new_state_dict = OrderedDict()
    for k, v in state_dict['net'].items():
        name = k[7:] # remove `module.`
        new_state_dict[name] = v
    # load params
    return new_state_dict
    # model.load_state_dict(new_state_dict)

def load_ablation_model(dataset, parallel=True, device='cuda'):
    assert dataset in ['MNIST', 'CIFAR']
    is_MNIST = (dataset=='MNIST')

    device = 'cuda'
    class Flatten(nn.Module):
        def forward(self, x):
            return x.view(x.size(0), -1)
    transform_test = transforms.Compose([
    transforms.ToTensor(),
    ])

    if is_MNIST: 
        model_path = 'mnist_lr_0.001_keep_45_epoch_399_resume_mnist_lr_0.01_keep_45_epoch_199.pth.pth'
        net = nn.Sequential(
                nn.Conv2d(2, 64, 4, stride=2, padding=1),
                nn.ReLU(),
                nn.Conv2d(64, 128, 4, stride=2, padding=1),
                nn.ReLU(),
                Flatten(),
                nn.Linear(128*7*7,500),
                nn.ReLU(),
                nn.Linear(500,100),
                nn.ReLU(),
                nn.Linear(100, 10)
            )
        net = net.to(device)
        keep = 45
        bs = 5
        # NOTE: DEFAULT VALUE 10000
        samples = 10000
        is_MNIST = True
        dataset = torchvision.datasets.MNIST(root=root+'/datasets/', train=False, download=True, transform=transform_test)
        loader = torch.utils.data.DataLoader(dataset, batch_size=bs, shuffle=False, num_workers=1)

    else:
        model_path = 'cifar_lr_0.001_regularization_0.0005_model_resnet18_keep_75_epoch_399_resume_cifar_lr_0.01_regularization_0.0005_model_resnet18_keep_75_epoch_199.pth.pth'
        net = resnet.ResNet18()
        net = net.to(device)
        keep = 75
        bs = 2
        # NOTE: DEFAULT VALUE 10000
        samples = 2000
        is_MNIST = False
        dataset = torchvision.datasets.CIFAR10(root=root+'/datasets/CIFAR/', train=False, download=True, transform=transform_test)
        loader = torch.utils.data.DataLoader(dataset, batch_size=bs, shuffle=False, num_workers=1)
    checkpoint_dir = root+'/randomizedAblation/'+'checkpoints'
    resume_file = '{}/{}'.format(checkpoint_dir, model_path)

    if parallel:
        net = torch.nn.DataParallel(net)
        checkpoint = torch.load(resume_file)
        net.load_state_dict(checkpoint['net'])
    else:
        converted_dict = convert_state_dict(resume_file)
        net.load_state_dict(converted_dict)    

    net.eval()
    cudnn.benchmark = True
    alpha = 0.05
    return net, loader, dataset, keep, alpha, bs, samples, is_MNIST

def predict(inputs, net, keep, samples, alpha, is_MNIST):
    if is_MNIST:
        predicted = utils.predict(inputs, net, keep, samples, alpha)
    else:
        predicted = utils_cifar.predict(inputs, net, keep, samples, alpha)
    
    return predicted

def evaluate(net, total, loader, keep, alpha, bs, samples, is_MNIST):
    
    num_batches = total//bs # samples to evaluate (assuming batch size is 1)
    device = 'cuda'

    # iniatilize 
    tot = 0
    correct = 0
    abstain = 0

    for batch_idx, (inputs, targets) in enumerate(loader):
        if batch_idx >= num_batches:
            break

        inputs, targets = inputs.to(device), targets.to(device)
        with torch.no_grad():
            predicted = predict(inputs, net, keep, samples, alpha, is_MNIST)
            correct += (predicted == targets.cpu()).sum()
            abstain += (predicted == -1).sum()
            tot += predicted.shape[0]
            progress_bar(batch_idx, len(loader), 'Acc: %.3f%% (%d/%d)' % (100.*correct/tot, correct, tot))
    out = {
        'total': tot,
        'correct': correct,
        'abstain': abstain
    }
    return out


In [4]:
from utils.attack import attack

net, loader, dataset, keep, alpha, _ , _, is_MNIST = load_ablation_model('MNIST')

bs = 5
data_x, data_y = [], []
data_iter = iter(loader)

for batch in range(20):
    x, y = data_iter.next()
    data_x.append(x)
    data_y.append(y)

In [5]:
budget = 12
beta = 1

r_acc, _, _, _ = attack(net, 
                        budget=budget, 
                        x=data_x,
                        y=data_y,
                        beta = beta,
                        n_queries=10000,
                        n_restarts=5,
                        device='cuda:0',
                        log_path='ablation_rs.log')

/home/mbeliaev/miniconda3/envs/robust/lib/python3.8/site-packages/scipy/stats/_discrete_distns.py:78: RuntimeWarning: divide by zero encountered in _binom_pdf
  return _boost._binom_pdf(x, n, p)
/home/mbeliaev/home/code/robust-l0/utils/attacks/sparse_rs.py:952: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /opt/conda/conda-bld/pytorch_1607370172916/work/torch/csrc/utils/python_arg_parser.cpp:882.)
  ind_to_fool = acc.nonzero().squeeze()


In [ ]:
r_acc

In [ ]:
# def run_pw_attack_ablation(fmodel, attack , dataset, num_iters, num_batches, device):
#     '''
#     Runs pointwise attack on model using x,y for ablation model
#     NOTE: assumes x,y is list of tensors 
#     '''
#     # run attack
#     advs = []
#     advs_dist = []
#     # for batch in trange(num_batches):
#     for batch in trange(num_batches):
#         (x, y) = dataset[batch]
#         x, y = x.unsqueeze(0).numpy(), np.array([y])
#         # x and y are individual samples here
#         adversarial_dists = []
#         adversarials = []
#         for _ in range(num_iters):
#             adv_x = attack(x, y, unpack=False)
#             distance = adv_x[0].distance.value
#             original_class = adv_x[0].original_class
#             output = adv_x[0].output
#             # distance = [x.distance.value for x in adv_x]
#             # original_class = [x.original_class for x in adv_x]
#             # output = [x.output for x in adv_x]
#             adversarial_dists.append(distance)
#             adversarials.append({'distance': distance,'label': original_class, 'scores':  output})

#         # find min
#         mindex = np.argmin(adversarial_dists)
#         advs.append(adversarials[mindex])
#         advs_dist.append(adversarial_dists[mindex])

#     advs_dist = np.asarray(advs_dist)
#     print('Final median l0 distance: ',np.median(advs_dist))

#     return advs, advs_dist



In [ ]:
# net, loader, dataset, keep, alpha, bs, samples, is_MNIST = load_ablation_model('CIFAR')
# # override samples
# samples = 10000
# # check clean accuracy using 20 samples|
# # out = evaluate(net, 20, loader, keep, alpha, bs, samples, is_MNIST) 
# # print(out)




In [ ]:
# total_samples = 5
# num_iters = 10

# fmodel = foolbox_utils.AblatedTorchModel(model=net,
#                                          num_samples=samples,
#                                          keep=keep,
#                                          bounds=(0, 1),
#                                          num_classes=10,
#                                          is_MNIST=is_MNIST)
# haattack = foolbox.attacks.PointwiseAttack(model=fmodel, 
#                                            criterion=foolbox_utils.MisclassificationOrAbstain(samples, alpha), 
#                                            distance=foolbox.distances.L0)

# advs, advs_dist = run_pw_attack_ablation(fmodel=fmodel,
#                                          attack=haattack,
#                                          dataset=dataset,
#                                          num_iters=num_iters,
#                                          num_batches=total_samples,
#                                          device='cuda')




